In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.1. Data Cleaning & Imputation/Output')
cur_date = "042623"

library(readxl)
library(tidyverse)
library(reshape2)
library(imputeLCMD)

# reading in files
mRNA_df = data.frame(read_excel("Input/mRNA_Data_041423.xlsx", sheet = 2))

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm

In [2]:
head(mRNA_df)

,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2
,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,1,F_1,ALDH1A3,C,F,1,24,-6.6824749,4.480105
2,2,M_2,ALDH1A3,C,F,1,24,0.2560009,4.870661
3,3,M_3,ALDH1A3,C,F,1,24,0.4318636,4.879307
4,4,F_4,ALDH1A3,C,F,1,24,0.4193253,4.878692
5,6,M_6,ALDH1A3,C,F,1,24,-0.1550695,4.850246
6,1,F_1,ALDH1A3,C,F,1,72,-0.2119125,4.847400


Identifying the number of replicates for each sample condition.

In [3]:
count_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Duplicate_Count = n()) %>% 
    arrange(-Duplicate_Count, mRNA, Subject_No, Condensate, Burn_Condition, Concentration, Time_Point) 

head(count_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Duplicate_Count
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,6
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,6
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


In [4]:
replicate_count_df = count_mRNA_df %>%
    group_by(Duplicate_Count) %>%
    count() %>%
    arrange(-Duplicate_Count) 

replicate_count_df

Duplicate_Count,n
<int>,<int>
6,6
5,80
4,328
3,96
2,1340
1,2361


In [5]:
# creating a new column for the replicate number for each condition
replicate_vector = c()
for (i in 1:length(replicate_count_df$n)){
    # calculating number of times to repeat the replicate number from 1 to the count
    replicate_count = replicate_count_df$n[i]/replicate_count_df$Duplicate_Count[i]
    replicate_iteration = rep(1:replicate_count_df$Duplicate_Count[i], times = replicate_count)
    
    # saving iteration
    replicate_vector = c(replicate_vector, replicate_iteration)
    
}

# adding to df 
count_mRNA_df$Replicate = replicate_vector
mRNA_df = count_mRNA_df %>%
    select(-Duplicate_Count)

head(mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,ddCT,ddCT_pslog2,Replicate
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
6,M_6,HMOX1,PBS,PBS,NA,24,0.70848207,4.892803,1
6,M_6,HMOX1,PBS,PBS,NA,24,0.36651641,4.876100,2
6,M_6,HMOX1,PBS,PBS,NA,24,-0.09090465,4.853452,3
6,M_6,HMOX1,PBS,PBS,NA,24,-0.37654696,4.839126,4
6,M_6,HMOX1,PBS,PBS,NA,24,0.72283828,4.893500,5
6,M_6,HMOX1,PBS,PBS,NA,24,0.37135970,4.876338,6


Take the average ddCT to get rid of the replicates.

In [6]:
avg_mRNA_df = mRNA_df %>%
    group_by(mRNA, Subject_ID, Condensate, Burn_Condition, Concentration, Time_Point) %>%
    mutate(Avg_ddCT_pslog2 = mean(ddCT_pslog2)) %>% #, Avg_ddCT = mean(ddCT)) %>%
    select(-c("Replicate", "ddCT_pslog2", "ddCT")) %>%
    unique()

head(avg_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Avg_ddCT_pslog2
<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
6,M_6,HMOX1,PBS,PBS,NA,24,4.871886
5,F_5,ALDH3A1,PBS,PBS,NA,4,4.878765
5,F_5,CXCL1,PBS,PBS,NA,4,4.889498
5,F_5,CXCR1,PBS,PBS,NA,4,4.049213
5,F_5,GCLC,PBS,PBS,NA,4,4.902343
5,F_5,GCLM,PBS,PBS,NA,4,4.874560


In [7]:
# converting to wide format to get missing values
preimputed_df = avg_mRNA_df %>%
    select(-Subject_No) %>%
    # creating sample ids
    unite("SampleID", c(colnames(mRNA_df)[c(3:7)])) %>%
    pivot_wider(names_from = "Subject_ID", values_from = "Avg_ddCT_pslog2") %>%
    pivot_longer(cols = 2:7, names_to = "Subject_ID", values_to = "Avg_ddCT_pslog2") %>%
    separate(SampleID, c("mRNA", "Condensate", "Burn_Condition", "Concentration", "Time_Point")) #%>%
    # adding a column for ????
    #mutate(new = 2^(-Avg_ddCT))


preimputed_df = preimputed_df[,c(6,1:5,7)]
head(preimputed_df)

Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Avg_ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
M_6,HMOX1,PBS,PBS,NA,24,4.871886
F_5,HMOX1,PBS,PBS,NA,24,NA
F_1,HMOX1,PBS,PBS,NA,24,NA
M_2,HMOX1,PBS,PBS,NA,24,NA
M_3,HMOX1,PBS,PBS,NA,24,NA
F_4,HMOX1,PBS,PBS,NA,24,NA


# QRILC Imputation

In [8]:
# QRILC
QRILC_imputation = function(dataset){
    # """
    # Creating a function to generate missing data
    # :param (input): df with missing data
    # :output: 1 imputed df
    # """
    wider_dataset = dataset %>%
        pivot_wider(names_from = mRNA, values_from = Avg_ddCT_pslog2) 
    
    # normalizing data since that's what the QRILC function wants
    QRILC_prep = wider_dataset[,6:dim(wider_dataset)[2]] %>%
         mutate_all(., function(x) log2(x)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log2_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log2_df %>%
        mutate_all(., function(x) 2^x)
    
    # adding back in ID col
    QRILC_df = cbind(wider_dataset[,1:5], QRILC_df)

    imputed_dataset = QRILC_df %>%
        pivot_longer(cols = 6:all_of(dim(wider_dataset)[2]), names_to = "mRNA", values_to = "ddCT_pslog2")

   return(imputed_dataset)
}

# calling fn
imputed_df = QRILC_imputation(preimputed_df)
head(imputed_df)

Warning message in (function (x) :
“NaNs produced”


Subject_ID,Condensate,Burn_Condition,Concentration,Time_Point,mRNA,ddCT_pslog2
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
M_6,PBS,PBS,NA,24,HMOX1,4.871886
M_6,PBS,PBS,NA,24,ALDH3A1,4.837012
M_6,PBS,PBS,NA,24,CXCL1,4.822372
M_6,PBS,PBS,NA,24,CXCR1,3.940873
M_6,PBS,PBS,NA,24,GCLC,4.863186
M_6,PBS,PBS,NA,24,GCLM,4.834127


In [9]:
# original data with replicates
dim(mRNA_df)

# after averaging replicates
dim(avg_mRNA_df)

# adding in missing data for remaining records
dim(preimputed_df)

dim(imputed_df)

[1] 4211   10

[1] 3162    8

[1] 3534    7

[1] 3534    7

In [10]:
# exporting data
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_mRNA.csv"), row.names = FALSE)